# IBM Data Science Professional Certificate Capstone Project

# Capstone Project - The Battle of Neighborhoods

### Introduction

Evey time I go abroad I have the same problem of which hotel to choose and which nice places there are around the hotel. I usually short of time and I need everything to be close to me in the best way possibe. I want to find the best hotels in the best locations. Everywhere.

### Business Problem

The searching procedure of hotels can be very frustrating. You don't realy know the city and where is the best places for you to stay depending on the quality of the hotel and the surroundings. It can take a lot of time to find a hotel which provides you the best deal for your stay. Everyone who is going to stay in a hotelcan be a potentional client of this project.

### Strategy

The aim of my work is to use machine learning in order to find the best hotels in the best areas and let the costumer to choose from a very limited collection of "the best of the best" what hotel is his number 1 hotel. This work will Use Foursquare API to get the Data of Hotels in any City provided by the User, it will display ratings of the hotels. So we can Rank Hotels. the code will also search for Nearby Places for the Top 5 Hotels and Display them to the user. It will help them to choosing the Hotel based on their Personal Favorable Nearby Places.

## Data

User will write down the name of the city and using Geopy Library, he will get it's coordinates. Using Folium library, a map of his chosen location will be made. On top of that map he will see the hotels and the surroundings.

Using Foursquare API the user will get hotels in a redius of 7 KM from his city of choice. A JSON file will be produced and it will be converted to Pandas DataFrame containing all the relevant hotels data.

Having hotel ID's, using foursquare API, the user will get ratings of the hotels in a data frame.

The next stage is to get top 5 by sorting them into a data frame.

Having the hotels locations, Foursquare API can find places in the area around the hotel. This data frame will be combined with the hotels data frame.

Finally, a map all Nearby Places along with the 5 Hotels will be generated with clusters of Hotels + Nearby Places using K-Means Algorithm.

Import the relevant libraries:

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

Check my desired city - Miami

In [3]:
address = 'Miami, Florida'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Miami are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Miami are 25.7742658, -80.1936589.


### 1. Check for hotels info at Foursquare

In [4]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'W1LVTWIPDLTONF0IUDBTE3ZW2KAG4NXB4MAXHLPWVFRXBJXZ' # your Foursquare ID
CLIENT_SECRET = '0FTT5AYQ3PH0N50PTR0IAUYZMIR35UVTDE5A1SK1XYT1H3GU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W1LVTWIPDLTONF0IUDBTE3ZW2KAG4NXB4MAXHLPWVFRXBJXZ
CLIENT_SECRET:0FTT5AYQ3PH0N50PTR0IAUYZMIR35UVTDE5A1SK1XYT1H3GU


In [5]:
# Search for hotels in Miami
search_query = 'hotel'
radius = 70000 # 7 KM
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
result_hotels = requests.get(url).json()
result_hotels

{'meta': {'code': 200, 'requestId': '5e68fd97be61c9001b07beb9'},
 'response': {'venues': [{'id': '4a9f2a6af964a520c43c20e3',
    'name': 'Kimpton EPIC Hotel',
    'location': {'address': '270 Biscayne Boulevard Way',
     'crossStreet': 'at SE 3rd Ave.',
     'lat': 25.770571,
     'lng': -80.189539,
     'labeledLatLngs': [{'label': 'display',
       'lat': 25.770571,
       'lng': -80.189539}],
     'distance': 582,
     'postalCode': '33131',
     'cc': 'US',
     'city': 'Miami',
     'state': 'FL',
     'country': 'United States',
     'formattedAddress': ['270 Biscayne Boulevard Way (at SE 3rd Ave.)',
      'Miami, FL 33131',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1583938961',
    'hasPerk': False},
   {'id': '

In [6]:
#create a map of Miami area:
Miami = folium.Map(location=[latitude, longitude], zoom_start=12)
   
Miami


Arrange the data:

In [7]:
# assign relevant part of JSON to hotel
hotels = result_hotels['response']['venues']

# tranform hotels into a dataframe
df_hotels = json_normalize(hotels)
df_hotels.shape

(30, 25)

In [8]:
# keep only columns that include hotel name, and anything that is associated with location:
filtered_columns = ['name', 'categories'] + [col for col in df_hotels.columns if col.startswith('location.')] + ['id']
df_hotels_filtered = df_hotels.loc[:, filtered_columns]

# function that extracts the category of the hotels
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_hotels_filtered['categories'] = df_hotels_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_hotels_filtered.columns = [column.split('.')[-1] for column in df_hotels_filtered.columns]

pd.DataFrame(df_hotels_filtered)
df_hotels_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Kimpton EPIC Hotel,Hotel,270 Biscayne Boulevard Way,US,Miami,United States,at SE 3rd Ave.,582,"[270 Biscayne Boulevard Way (at SE 3rd Ave.), ...","[{'label': 'display', 'lat': 25.770571, 'lng':...",25.770571,-80.189539,NaN,33131,FL,4a9f2a6af964a520c43c20e3
1,Four Seasons Hotel Miami,Hotel,1435 Brickell Ave,US,Miami,United States,at SE 14th Terr,1700,"[1435 Brickell Ave (at SE 14th Terr), Miami, F...","[{'label': 'display', 'lat': 25.75907805851922...",25.759078,-80.191875,NaN,33131,FL,4b069b0ff964a520a6ed22e3
2,Hotel Beaux Arts Miami,Hotel,"255 Biscayne Boulevard Way, Located Inside Of ...",US,Miami,United States,NaN,536,"[255 Biscayne Boulevard Way, Located Inside Of...","[{'label': 'display', 'lat': 25.77123917285659...",25.771239,-80.189500,NaN,33131,FL,4d6ecdeb7c726dcb9cb4811d
3,YVE Hotel Miami,Hotel,146 Biscayne Blvd,US,Miami,United States,NaN,571,"[146 Biscayne Blvd, Miami, FL 33132, United St...","[{'label': 'display', 'lat': 25.77567, 'lng': ...",25.775670,-80.188180,NaN,33132,FL,5842240b45005e59fe0281a4
4,Intercontinental Hotel Pool Deck,Hotel Pool,1000 Chopin Place,US,Miami,United States,NaN,880,"[1000 Chopin Place, Miami, FL, United States]","[{'label': 'display', 'lat': 25.77208843006392...",25.772088,-80.185219,Miami Central Business District,NaN,FL,52d9d3b4498e2904745ee857
5,SLS Brickell Hotel & Residence,Hotel,1300 S Miami Ave,US,Miami,United States,NaN,1457,"[1300 S Miami Ave, Miami, FL 33130, United Sta...","[{'label': 'display', 'lat': 25.76117006082683...",25.761170,-80.193651,NaN,33130,FL,5997438b12138410b7334d89
6,Hotel YVE Miami,Hotel,146 Biscayne Blvd,US,Miami,United States,NaN,591,"[146 Biscayne Blvd, Miami, FL 33132, United St...","[{'label': 'display', 'lat': 25.77620941352356...",25.776209,-80.188169,NaN,33132,FL,5489a637498efbecf5aa961b
7,Seminole Hard Rock Hotel & Casino,Hotel,1 Seminole Way,US,Hollywood,United States,NaN,31005,"[1 Seminole Way, Hollywood, FL 33314, United S...","[{'label': 'display', 'lat': 26.05232988433903...",26.052330,-80.211568,NaN,33314,FL,40e0b100f964a5203e051fe3
8,1 Hotel South Beach,Hotel,2341 Collins Ave,US,Miami Beach,United States,NaN,7229,"[2341 Collins Ave, Miami Beach, FL 33139, Unit...","[{'label': 'display', 'lat': 25.79916461326906...",25.799165,-80.127046,NaN,33139,FL,529f5c7f11d25082caffbb26
9,Fortune House Hotel,Hotel,185 SE 14th Ter,US,Miami,United States,at Brickell Ave.,1674,"[185 SE 14th Ter (at Brickell Ave.), Miami, FL...","[{'label': 'display', 'lat': 25.7594199, 'lng'...",25.759420,-80.190957,NaN,33131,FL,4df9639b1f6e2728c9efa839


In [9]:
# drop unrelevant columns:
df_hotels_filtered.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city'], axis=1, inplace=True)
df_hotels_filtered.head()

,name,categories,address,cc,distance,formattedAddress,lat,lng,neighborhood,id
0,Kimpton EPIC Hotel,Hotel,270 Biscayne Boulevard Way,US,582,"[270 Biscayne Boulevard Way (at SE 3rd Ave.), ...",25.770571,-80.189539,NaN,4a9f2a6af964a520c43c20e3
1,Four Seasons Hotel Miami,Hotel,1435 Brickell Ave,US,1700,"[1435 Brickell Ave (at SE 14th Terr), Miami, F...",25.759078,-80.191875,NaN,4b069b0ff964a520a6ed22e3
2,Hotel Beaux Arts Miami,Hotel,"255 Biscayne Boulevard Way, Located Inside Of ...",US,536,"[255 Biscayne Boulevard Way, Located Inside Of...",25.771239,-80.189500,NaN,4d6ecdeb7c726dcb9cb4811d
3,YVE Hotel Miami,Hotel,146 Biscayne Blvd,US,571,"[146 Biscayne Blvd, Miami, FL 33132, United St...",25.775670,-80.188180,NaN,5842240b45005e59fe0281a4
4,Intercontinental Hotel Pool Deck,Hotel Pool,1000 Chopin Place,US,880,"[1000 Chopin Place, Miami, FL, United States]",25.772088,-80.185219,Miami Central Business District,52d9d3b4498e2904745ee857


###### Visualize the hotels on a map

In [10]:
# create map of Miami using latitude and longitude values:
map_miami = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_hotels_filtered['lat'], df_hotels_filtered['lng'], df_hotels_filtered['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_miami)  
    
map_miami

## Methodology

Our aim is to search for the best hotels in a 7 KM Radius from Miami coordinates

We have collected the required data: location(coordinates) and type (category) of every Hotel (From Foursquare API).

Next, using foursquare API, we will gather hotels rating and focus on top 5 and their surroundings.

In third and final step we will focus on most promising areas and within those create clusters of locations that meet some basic requirements established in discussion with stakeholders: We will Cluster Hotel and their Nearby Places using K-means Clustering.

### 2. Get Ratings for All Hotels

Using For Loop we will Iterate to every Hotels and Fetch Corresponding Ratings and then Store it in new dataframe ratings

In [11]:
ratings = pd.DataFrame()
list1 = pd.Series([]) 
list2 = pd.Series([]) 
for i in range(len(df_hotels_filtered.id)):
    hotel_id = df_hotels_filtered.id[i] # ID of Hotels
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(hotel_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    #Let's Check for Ratings, If Available then store it and if not, then store 0
    try:
        list1[i] = result['response']['venue']['rating'] 
        list2[i] = df_hotels_filtered.id[i]
    except:
        list1[i] = 0
        list2[i] = df_hotels_filtered.id[i]
         
        
ratings.insert(0, "ID", list2, allow_duplicates=True)
ratings.insert(1, "Ratings", list1, allow_duplicates=True)
ratings

,ID,Ratings
0,4a9f2a6af964a520c43c20e3,8.9
1,4b069b0ff964a520a6ed22e3,9.3
2,4d6ecdeb7c726dcb9cb4811d,6.9
3,5842240b45005e59fe0281a4,0.0
4,52d9d3b4498e2904745ee857,0.0
5,5997438b12138410b7334d89,0.0
6,5489a637498efbecf5aa961b,6.5
7,40e0b100f964a5203e051fe3,8.9
8,529f5c7f11d25082caffbb26,9.4
9,4df9639b1f6e2728c9efa839,7.1


In [12]:
#merge both dataframes based on id
ratings.columns = ['id', 'Ratings']
df_hotels_filtered = pd.merge(df_hotels_filtered,
                 ratings,
                 on='id')
df_hotels_filtered

,name,categories,address,cc,distance,formattedAddress,lat,lng,neighborhood,id,Ratings
0,Kimpton EPIC Hotel,Hotel,270 Biscayne Boulevard Way,US,582,"[270 Biscayne Boulevard Way (at SE 3rd Ave.), ...",25.770571,-80.189539,NaN,4a9f2a6af964a520c43c20e3,8.9
1,Four Seasons Hotel Miami,Hotel,1435 Brickell Ave,US,1700,"[1435 Brickell Ave (at SE 14th Terr), Miami, F...",25.759078,-80.191875,NaN,4b069b0ff964a520a6ed22e3,9.3
2,Hotel Beaux Arts Miami,Hotel,"255 Biscayne Boulevard Way, Located Inside Of ...",US,536,"[255 Biscayne Boulevard Way, Located Inside Of...",25.771239,-80.189500,NaN,4d6ecdeb7c726dcb9cb4811d,6.9
3,YVE Hotel Miami,Hotel,146 Biscayne Blvd,US,571,"[146 Biscayne Blvd, Miami, FL 33132, United St...",25.775670,-80.188180,NaN,5842240b45005e59fe0281a4,0.0
4,Intercontinental Hotel Pool Deck,Hotel Pool,1000 Chopin Place,US,880,"[1000 Chopin Place, Miami, FL, United States]",25.772088,-80.185219,Miami Central Business District,52d9d3b4498e2904745ee857,0.0
5,SLS Brickell Hotel & Residence,Hotel,1300 S Miami Ave,US,1457,"[1300 S Miami Ave, Miami, FL 33130, United Sta...",25.761170,-80.193651,NaN,5997438b12138410b7334d89,0.0
6,Hotel YVE Miami,Hotel,146 Biscayne Blvd,US,591,"[146 Biscayne Blvd, Miami, FL 33132, United St...",25.776209,-80.188169,NaN,5489a637498efbecf5aa961b,6.5
7,Seminole Hard Rock Hotel & Casino,Hotel,1 Seminole Way,US,31005,"[1 Seminole Way, Hollywood, FL 33314, United S...",26.052330,-80.211568,NaN,40e0b100f964a5203e051fe3,8.9
8,1 Hotel South Beach,Hotel,2341 Collins Ave,US,7229,"[2341 Collins Ave, Miami Beach, FL 33139, Unit...",25.799165,-80.127046,NaN,529f5c7f11d25082caffbb26,9.4
9,Fortune House Hotel,Hotel,185 SE 14th Ter,US,1674,"[185 SE 14th Ter (at Brickell Ave.), Miami, FL...",25.759420,-80.190957,NaN,4df9639b1f6e2728c9efa839,7.1


#### Lets Remove Hotels with No Ratings

In [13]:
df_hotels_filtered = df_hotels_filtered.set_index("Ratings")
df_hotels_filtered = df_hotels_filtered.drop(0.0, axis=0) # Delete all rows with Ratings 0
df_hotels_filtered

,name,categories,address,cc,distance,formattedAddress,lat,lng,neighborhood,id
Ratings,,,,,,,,,,
8.9,Kimpton EPIC Hotel,Hotel,270 Biscayne Boulevard Way,US,582,"[270 Biscayne Boulevard Way (at SE 3rd Ave.), ...",25.770571,-80.189539,NaN,4a9f2a6af964a520c43c20e3
9.3,Four Seasons Hotel Miami,Hotel,1435 Brickell Ave,US,1700,"[1435 Brickell Ave (at SE 14th Terr), Miami, F...",25.759078,-80.191875,NaN,4b069b0ff964a520a6ed22e3
6.9,Hotel Beaux Arts Miami,Hotel,"255 Biscayne Boulevard Way, Located Inside Of ...",US,536,"[255 Biscayne Boulevard Way, Located Inside Of...",25.771239,-80.189500,NaN,4d6ecdeb7c726dcb9cb4811d
6.5,Hotel YVE Miami,Hotel,146 Biscayne Blvd,US,591,"[146 Biscayne Blvd, Miami, FL 33132, United St...",25.776209,-80.188169,NaN,5489a637498efbecf5aa961b
8.9,Seminole Hard Rock Hotel & Casino,Hotel,1 Seminole Way,US,31005,"[1 Seminole Way, Hollywood, FL 33314, United S...",26.052330,-80.211568,NaN,40e0b100f964a5203e051fe3
9.4,1 Hotel South Beach,Hotel,2341 Collins Ave,US,7229,"[2341 Collins Ave, Miami Beach, FL 33139, Unit...",25.799165,-80.127046,NaN,529f5c7f11d25082caffbb26
7.1,Fortune House Hotel,Hotel,185 SE 14th Ter,US,1674,"[185 SE 14th Ter (at Brickell Ave.), Miami, FL...",25.759420,-80.190957,NaN,4df9639b1f6e2728c9efa839
8.3,The Langford Hotel,Hotel,NaN,US,231,"[Miami, FL 33131, United States]",25.773547,-80.191493,Miami Central Business District,4f625e57e4b0ed0158428683
9.0,Loews Miami Beach Hotel,Hotel,1601 Collins Ave,US,6677,"[1601 Collins Ave (at 16th St), Miami Beach, F...",25.789474,-80.129216,NaN,40e0b100f964a520eb041fe3


In [14]:
#Sorting the Ratings
df_hotels_filtered.sort_values('Ratings', ascending=False, inplace=True) 

#Make new dataframe for top5 hotels
df_top5_hotels = df_hotels_filtered.head(5)
df_top5_hotels

,name,categories,address,cc,distance,formattedAddress,lat,lng,neighborhood,id
Ratings,,,,,,,,,,
9.4,1 Hotel South Beach,Hotel,2341 Collins Ave,US,7229,"[2341 Collins Ave, Miami Beach, FL 33139, Unit...",25.799165,-80.127046,NaN,529f5c7f11d25082caffbb26
9.3,Four Seasons Hotel Miami,Hotel,1435 Brickell Ave,US,1700,"[1435 Brickell Ave (at SE 14th Terr), Miami, F...",25.759078,-80.191875,NaN,4b069b0ff964a520a6ed22e3
9.1,SLS Hotel South Beach,Hotel,"1701 Collins Ave, Miami Beach",US,6783,"[1701 Collins Ave, Miami Beach (at 17th St), M...",25.792589,-80.129112,NaN,4fcd4704e4b00c762cbcb75e
9.0,Loews Miami Beach Hotel,Hotel,1601 Collins Ave,US,6677,"[1601 Collins Ave (at 16th St), Miami Beach, F...",25.789474,-80.129216,NaN,40e0b100f964a520eb041fe3
8.9,Kimpton EPIC Hotel,Hotel,270 Biscayne Boulevard Way,US,582,"[270 Biscayne Boulevard Way (at SE 3rd Ave.), ...",25.770571,-80.189539,NaN,4a9f2a6af964a520c43c20e3


#### Show top 5 hotels in Miami on map:

In [15]:
# create map of Miami using latitude and longitude values
top5_miami = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(df_top5_hotels['lat'], df_top5_hotels['lng'], df_top5_hotels['name']):
    label = '{}'.format(label)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(top5_miami)  
    
top5_miami

## 3. Search for Nearby Places

###### First, we have created blank Dictionary 'd' to store multiple dataframes, Each Dataframe contains 10 Nearby Places under the radius of 500 Meters from the Hotel.
###### Then, using For Loop we can iterate to Every Hotel's Co-ordinates and search for Nearby Places. then Finally filtering Columns we will store Dataframe in Dictionary

In [16]:
d = {}
for i in range(len(df_top5_hotels)):
    d[i] = pd.DataFrame()
    lat = df_top5_hotels['lat'].iloc[i]
    lng = df_top5_hotels['lng'].iloc[i]
    LIMIT=10
    radius = 500
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&near={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    items = results['response']['groups'][0]['items']
    dataframe = json_normalize(items) # flatten JSON
      
    # filter columns
    filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
    df = dataframe.loc[:, filtered_columns]
    
    # filter the category for each row
    df['venue.categories'] = df.apply(get_category_type, axis=1)
    
    # clean columns
    df.columns = [col.split('.')[-1] for col in df.columns]
    
    d[i] = df

In [17]:
#Extracting Dataframes from 'd' and storing into new dataframe for each hotels
df_hotel1 = d[0]
df_hotel2 = d[1]
df_hotel3 = d[2]
df_hotel4 = d[3]
df_hotel5 = d[4]

In [18]:
df_hotel1

,name,categories,address,cc,city,country,crossStreet,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,1 Hotel South Beach,Hotel,2341 Collins Ave,US,Miami Beach,United States,NaN,"[2341 Collins Ave, Miami Beach, FL 33139, Unit...","[{'label': 'display', 'lat': 25.79916461326906...",25.799165,-80.127046,NaN,33139,FL,529f5c7f11d25082caffbb26
1,W South Beach,Hotel,2201 Collins Ave,US,Miami Beach,United States,at 22nd St.,"[2201 Collins Ave (at 22nd St.), Miami Beach, ...","[{'label': 'display', 'lat': 25.79738125098067...",25.797381,-80.127190,NaN,33139,FL,4abedc44f964a5204f9020e3
2,1 Hotel South Beach Rooftop & Lounge Bar,Roof Deck,2341 Collins Ave,US,Miami Beach,United States,NaN,"[2341 Collins Ave, Miami Beach, FL 33139, Unit...","[{'label': 'display', 'lat': 25.7995344739414,...",25.799534,-80.126862,NaN,33139,FL,558625e8498e7cd696c5c91e
3,Ola Restaurant,Latin American Restaurant,1745 James Ave,US,Miami Beach,United States,NaN,"[1745 James Ave, Miami Beach, FL 33139, United...","[{'label': 'display', 'lat': 25.799417, 'lng':...",25.799417,-80.127905,City Center,33139,FL,4a3582f4f964a520149d1fe3
4,24th Street Beach,Beach,Collins Ave,US,Miami Beach,United States,24th St,"[Collins Ave (24th St), Miami Beach, FL 33139,...","[{'label': 'display', 'lat': 25.79981468309582...",25.799815,-80.125496,NaN,33139,FL,4d1cfb64e56f6ea8ef9a571d
5,LivingRoom Lounge At The W SouthBeach,Bar,2201 Collins Ave,US,Miami Beach,United States,22nd St,"[2201 Collins Ave (22nd St), Miami Beach, FL 3...","[{'label': 'display', 'lat': 25.79743236478577...",25.797432,-80.127158,NaN,33139,FL,4ec31a7d77c83d76b137f610
6,WET - W South Beach,Hotel Pool,2201 Collins Ave,US,Miami Beach,United States,at 22nd St,"[2201 Collins Ave (at 22nd St), Miami Beach, F...","[{'label': 'display', 'lat': 25.79744854458237...",25.797449,-80.126841,NaN,33139,FL,4bc0b32cf8219c7444feb110
7,Villa Azur Restaurant and Lounge,French Restaurant,309 23rd St,US,Miami Beach,United States,NaN,"[309 23rd St, Miami Beach, FL 33139, United St...","[{'label': 'display', 'lat': 25.79903656378283...",25.799037,-80.129186,NaN,33139,FL,4f298d70e4b0a2a2c413d50f
8,SoulCycle South Beach,Cycle Studio,2325 Collins Ave,US,Miami,United States,NaN,"[2325 Collins Ave, Miami, FL 33139, United Sta...","[{'label': 'display', 'lat': 25.79893018400649...",25.798930,-80.127745,NaN,33139,FL,565c6773498e0edfdcfb201f
9,STK Miami,Steakhouse,2311 Collins Ave,US,Miami Beach,United States,Collins,"[2311 Collins Ave (Collins), Miami Beach, FL 3...","[{'label': 'display', 'lat': 25.79862236168264...",25.798622,-80.127894,NaN,33139,FL,55020e0b498e96fb35e7caa1


In [19]:
# let's clean unwanted columns:

df_hotel1.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)
df_hotel2.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)
df_hotel3.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)
df_hotel4.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)
df_hotel5.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city', 'neighborhood', 'address', 'cc'], axis=1, inplace=True)

In [20]:
df_hotel5.head()

,name,categories,formattedAddress,lat,lng,id
0,Whole Foods Market,Grocery Store,"[299 SE 3rd Ave (at SE 3rd St), Miami, FL 3313...",25.772000,-80.189146,515989a3e4b03ece5bbf55a0
1,The Capital Grille,American Restaurant,"[444 Brickell Ave, Miami, FL 33131, United Sta...",25.769391,-80.190237,4a91e72bf964a5204c1c20e3
2,Zuma,Japanese Restaurant,"[270 Biscayne Boulevard Way, Miami, FL 33131, ...",25.769441,-80.189441,4bf33b04cad2c928c2a29a99
3,Icon Brickell,Residential Building (Apartment / Condo),"[465 Brickell Ave, Miami, FL 33131, United Sta...",25.768766,-80.189193,4b5b8545f964a520560429e3
4,JW Marriott Marquis Miami,Hotel,"[255 Biscayne Boulevard Way (33131), Miami, FL...",25.771089,-80.189448,4d2504cc38d0236a0958e8b6


In [21]:
# We have to Remove Special Characters from dataframe inorder to use Place names as a label on map

#replace (') with blank 
df_hotel1 = df_hotel1.replace('\'','',regex=True)
df_hotel2 = df_hotel2.replace('\'','',regex=True)
df_hotel3 = df_hotel3.replace('\'','',regex=True)
df_hotel4 = df_hotel4.replace('\'','',regex=True)
df_hotel5 = df_hotel5.replace('\'','',regex=True)

In [22]:
df_hotel5.head()

,name,categories,formattedAddress,lat,lng,id
0,Whole Foods Market,Grocery Store,"[299 SE 3rd Ave (at SE 3rd St), Miami, FL 3313...",25.772000,-80.189146,515989a3e4b03ece5bbf55a0
1,The Capital Grille,American Restaurant,"[444 Brickell Ave, Miami, FL 33131, United Sta...",25.769391,-80.190237,4a91e72bf964a5204c1c20e3
2,Zuma,Japanese Restaurant,"[270 Biscayne Boulevard Way, Miami, FL 33131, ...",25.769441,-80.189441,4bf33b04cad2c928c2a29a99
3,Icon Brickell,Residential Building (Apartment / Condo),"[465 Brickell Ave, Miami, FL 33131, United Sta...",25.768766,-80.189193,4b5b8545f964a520560429e3
4,JW Marriott Marquis Miami,Hotel,"[255 Biscayne Boulevard Way (33131), Miami, FL...",25.771089,-80.189448,4d2504cc38d0236a0958e8b6


In [23]:
# In order to have a better look at the center of interest, I add 0.04 to the longitud
miami_hotels_map = folium.Map(location=[latitude, longitude+0.04], zoom_start=14) # generate map centred around Miami

# add a marker to represent Hotel1
folium.Marker(
    [df_top5_hotels['lat'].iloc[0], df_top5_hotels['lng'].iloc[0]],
    popup=df_top5_hotels['name'].iloc[0],
    ).add_to(miami_hotels_map)

# add the Nearby Places as blue circle markers
for lat, lng, label in zip(df_hotel1['lat'], df_hotel1['lng'], df_hotel1['name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(miami_hotels_map)
    
# add a marker to represent Hotel2
folium.Marker(
    [df_top5_hotels['lat'].iloc[1], df_top5_hotels['lng'].iloc[1]],
    popup=df_top5_hotels['name'].iloc[1],
    ).add_to(miami_hotels_map)

# add the Nearby Places as green circle markers
for lat1, lng1, label1 in zip(df_hotel2['lat'], df_hotel2['lng'], df_hotel2['name']):
    folium.features.CircleMarker(
        [lat1, lng1],
        radius=5,
        color='green',
        popup=label1,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
        ).add_to(miami_hotels_map)
    
# add a marker to represent Hotel3
folium.Marker(
    [df_top5_hotels['lat'].iloc[2], df_top5_hotels['lng'].iloc[2]],
    popup=df_top5_hotels['name'].iloc[2],
    ).add_to(miami_hotels_map)

# add the Nearby Places as orange circle markers
for lat, lng, label in zip(df_hotel3['lat'], df_hotel3['lng'], df_hotel3['name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.6
        ).add_to(miami_hotels_map)
    
# add a marker to represent Hotel4
folium.Marker(
    [df_top5_hotels['lat'].iloc[3], df_top5_hotels['lng'].iloc[3]],
    popup=df_top5_hotels['name'].iloc[3],
    ).add_to(miami_hotels_map)

# add the Nearby Places as purple circle markers
for lat, lng, label in zip(df_hotel4['lat'], df_hotel4['lng'], df_hotel4['name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='purple',
        popup=label,
        fill = True,
        fill_color='purple',
        fill_opacity=0.6
        ).add_to(miami_hotels_map)
    
# add a marker to represent Hotel5
folium.Marker(
    [df_top5_hotels['lat'].iloc[4], df_top5_hotels['lng'].iloc[4]],
    popup=df_top5_hotels['name'].iloc[4],
    ).add_to(miami_hotels_map)

# add the Nearby Places as red circle markers
for lat, lng, label in zip(df_hotel5['lat'], df_hotel5['lng'], df_hotel5['name']):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
        ).add_to(miami_hotels_map)

# display map
miami_hotels_map

## 4. Clustering Hotels with Nearby Places using K-Means

In [24]:
#Combine all dataframe and store it in df_nearbyplaces then reset the index values
df_nearbyplaces = pd.concat([df_hotel1, df_hotel2, df_hotel3, df_hotel4, df_hotel5], axis=0).reset_index(drop=True)
df_nearbyplaces.head()

,name,categories,formattedAddress,lat,lng,id
0,1 Hotel South Beach,Hotel,"[2341 Collins Ave, Miami Beach, FL 33139, Unit...",25.799165,-80.127046,529f5c7f11d25082caffbb26
1,W South Beach,Hotel,"[2201 Collins Ave (at 22nd St.), Miami Beach, ...",25.797381,-80.127190,4abedc44f964a5204f9020e3
2,1 Hotel South Beach Rooftop & Lounge Bar,Roof Deck,"[2341 Collins Ave, Miami Beach, FL 33139, Unit...",25.799534,-80.126862,558625e8498e7cd696c5c91e
3,Ola Restaurant,Latin American Restaurant,"[1745 James Ave, Miami Beach, FL 33139, United...",25.799417,-80.127905,4a3582f4f964a520149d1fe3
4,24th Street Beach,Beach,"[Collins Ave (24th St), Miami Beach, FL 33139,...",25.799815,-80.125496,4d1cfb64e56f6ea8ef9a571d


In [25]:
number_of_clusters = 5 #define number of cluster to be generated. In our case it is 5 because we have 5 hotels

loc = df_nearbyplaces[['lat', 'lng']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(loc)

labels = kmeans.labels_

In [26]:
df_nearbyplaces["Labels"] = labels #adding cluster labels to main dataframe
df_nearbyplaces.head()

,name,categories,formattedAddress,lat,lng,id,Labels
0,1 Hotel South Beach,Hotel,"[2341 Collins Ave, Miami Beach, FL 33139, Unit...",25.799165,-80.127046,529f5c7f11d25082caffbb26,3
1,W South Beach,Hotel,"[2201 Collins Ave (at 22nd St.), Miami Beach, ...",25.797381,-80.127190,4abedc44f964a5204f9020e3,3
2,1 Hotel South Beach Rooftop & Lounge Bar,Roof Deck,"[2341 Collins Ave, Miami Beach, FL 33139, Unit...",25.799534,-80.126862,558625e8498e7cd696c5c91e,3
3,Ola Restaurant,Latin American Restaurant,"[1745 James Ave, Miami Beach, FL 33139, United...",25.799417,-80.127905,4a3582f4f964a520149d1fe3,3
4,24th Street Beach,Beach,"[Collins Ave (24th St), Miami Beach, FL 33139,...",25.799815,-80.125496,4d1cfb64e56f6ea8ef9a571d,3


In [27]:
cluster_centers = pd.DataFrame(kmeans.cluster_centers_)
df_nearbyplaces['Labels'].unique()

array([3, 2, 1, 4, 0])

#### Let's Visualize Our Final Data on Map with Clusters

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude+0.04], zoom_start=14) # generate map centred around Miami

# set color scheme for the clusters
x = np.arange(number_of_clusters)
ys = [i + x + (i*x)**2 for i in range(number_of_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add Cluster circles to the map
for lon, lat in zip(cluster_centers[0], cluster_centers[1]):
    folium.CircleMarker([lon, lat], radius=50, color='yellow', fill=True, fill_opacity=0.25).add_to(map_clusters)
    folium.Marker(
    [lon, lat],
    popup='Cluster Center',
    ).add_to(map_clusters)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_nearbyplaces['lat'], df_nearbyplaces['lng'], df_nearbyplaces['name'], df_nearbyplaces['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Results:

This analysis Shows that it is possible to get information about hotels in Miami, and retriving ratings not of all of them. Some have no rating at all.
All hotels were sorted based on ratings and Top 5 hotels were selected to recommend them to the user.
Nearby places from selected 5 hotels were searched. From this data we can see that we have various categories of places that users can choose hotel based on having their interested nearby places to visit and enjoy.
In the last step, clusters of best hotels and their surroundings were made for the costumer's convenience

## Discussion:

People choose hotels also by their price per night and by their facilities. Moreover, the proximity to central bus/train stations can also be an issue plus many more features.
Here there is a basic analysis. A deeper analysis can and must be made in order to get the best costumer satisfaction.
For example, a list of restaurants of the costumer choice can be shown, museums, stores and more...

## Conclusion:

In this project a recommendation of the Best Hotels in a City together with nearby places was achived. The aim was to help Tourists in narrowing down the search for optimal location for stay. By getting ratings of All Hotels from Foursquare data, Top 5 hotels were sorted and a collection of nearby places which satisfy some basic requirements regarding favourite Places was added. Clustering of those nearby places was then performed in order to create major zones of interest (containing nearby places) and addresses of those zone centers were created to be used as starting points for exploration by the costumers.

The final decission will be made by the costumers based on specific Requirements in every recommended zone. the costumer will choose and select the hotel to stay in by his own needs. This project can provide the costumer more than enough information in order to help him choose the best possible hotel in every city.



# Hope you enjoyed :)